_This page is part of the material for the ["Introduction to Tensorflow"](https://indico.cism.ucl.ac.be/event/84/) session of the [2020 CISM/CÉCI trainings](http://www.ceci-hpc.be/training.html), see the [table of contents](index.html) for the other parts. The notebook can be downloaded with [this link](mlintro.ipynb)._                   

Tensorflow is a widely used library for machine learning, especially deep learning, both training and inference (evaluating trained neural networks on new data).
It was developed by the Google Brain team, and is open source software.
On the [website](https://www.tensorflow.org) you will find many libraries and tools for common tasks related to machine learning, and a lot of training material and examples.

In this second part, we will look in a bit more detail at how [Tensorflow](https://www.tensorflow.org) works at a technical level, and why that makes it such an interesting choice.

## Fundamental classes and concepts

In the [first example](mlintro.html#A-first-example) we have used Tensorflow largely as a drop-in replacement.
That worked because it provides a lot of the same interface, and because the `Tensor` class can for many things be used just like a [numpy](https://numpy.org) array.

In [1]:
import tensorflow as tf
x1 = tf.linspace(0., 1., 11)
print(x1)

tf.Tensor(
[0.         0.1        0.2        0.3        0.4        0.5
 0.6        0.7        0.8        0.90000004 1.        ], shape=(11,), dtype=float32)


In fact, `Tensor` is the class used for almost everything with numerical values in Tensorflow: a single number is a 0-dimensional tensor, a list (like above) a 1-dimensional tensor, a matrix a 2-dimensional tensor etc.
To construct a `Tensor` from its value(s), the `tf.constant` helper method can be used:

In [2]:
x0 = tf.constant(3.14)
x2 = tf.constant([ [ 1., 2.], [3., 4. ] ])
print(x1)
print(x2)

tf.Tensor(
[0.         0.1        0.2        0.3        0.4        0.5
 0.6        0.7        0.8        0.90000004 1.        ], shape=(11,), dtype=float32)
tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


As you can see above, the shape of the tensor is a tuple with the number of elements in each dimension, and the default type a 32-bit floating point number; if we were to construct one from integers, we would get a 32-bit integer type:

In [3]:
x3 = tf.constant([ 1, 1, 2, 3, 5, 8, 13, 25 ])
print(x3)

tf.Tensor([ 1  1  2  3  5  8 13 25], shape=(8,), dtype=int32)


These are all tensors with a static shape, like in numpy, but it is also possible to make tensors with dynamic shape, if the size in one dimension is not known beforehand.
This is used for input nodes, which can be constructed before the batch size is known:

In [4]:
x4 = tf.keras.layers.Input(shape=(3,))
print(x4)

Tensor("input_1:0", shape=(None, 3), dtype=float32)


It is also possible to make tensors where one dimension changes from entry to entry, so `Tensor` provides an extension of a numpy array.

There is one more important feature about tensors, and this is actually the original reason they were developed: they can be used to construct a *computation graph*, which represents the calculations that need to be done, without actually performing them.
Since Tensorflow 2.0, the default changed to use the "eager" mode, where the calculation is performed immediately, but constructing graphs (and optimizing them) provides better performance, so this is used by the higher-level helper functions to construct neural networks.

It requires a bit of setup, but we can use [Tensorboard](https://www.tensorflow.org/tensorboard) can be used to visualize such graphs (the example is taken from [here](https://www.tensorflow.org/tensorboard/graphs#graphs_of_tffunctions) in the documentation).

In [5]:
%load_ext tensorboard
# the following line tells the extension where to find tensorboard
# you will either not need, or have to change the path
# To find the path: activate the conda / virtual environment, and run `which tensorboard`
%env TENSORBOARD_BINARY=/home/pieter/miniconda3/envs/ceci_mltf/bin/tensorboard

env: TENSORBOARD_BINARY=/home/pieter/miniconda3/envs/ceci_mltf/bin/tensorboard


In [6]:
# 3x3 random matrics
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

@tf.function
def f1(a, b):
    return tf.nn.relu(tf.matmul(a, b))

tf.summary.trace_on(graph=True, profiler=True)
z = f1(x, y)
from datetime import datetime
logdir = f'logs/func/{datetime.now().strftime("%Y%m%d-%H%M%S")}'
with tf.summary.create_file_writer(logdir).as_default():
    tf.summary.trace_export(
        name="f1_trace_xy",
        step=0,
        profiler_outdir=logdir)

%tensorboard --logdir logs/func

Reusing TensorBoard on port 6007 (pid 22213), started 0:01:15 ago. (Use '!kill 22213' to kill it.)

In [6]:
y1 = tf.Variable(3.14)
print(y1)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.14>
